# 多量子门、受控门操作(CommonMutilGates Operations)
包括多量子门、受控门操作


两个量子位的表示可以由两个量子态的直积结果表示。首先给定两个量子态：
$$|\psi_1\rangle = \begin{bmatrix} \alpha \\ \beta \end{bmatrix}，|\psi_2\rangle = \begin{bmatrix} \gamma \\ \delta \end{bmatrix}$$

其中 $\alpha, \beta, \gamma, \delta$ 是复数，满足归一化条件 $|\alpha|^2 + |\beta|^2 = 1$ 和 $|\gamma|^2 + |\delta|^2 = 1$。


两个单量子比特态的张量积表示为：

$$|\psi_1\rangle \otimes |\psi_2\rangle = \begin{bmatrix} \alpha \\ \beta \end{bmatrix} \otimes \begin{bmatrix} \gamma \\ \delta \end{bmatrix} = \begin{bmatrix} \alpha \cdot \gamma \\ \alpha \cdot \delta \\ \beta \cdot \gamma \\ \beta \cdot \delta \end{bmatrix}$$

容易验证其对归一化的满足和对于四种观测结果的一一对应，即
1. $|00\rangle = |0\rangle \otimes |0\rangle$
   $$|00\rangle = \begin{bmatrix} 1 \\ 0 \end{bmatrix} \otimes \begin{bmatrix} 1 \\ 0 \end{bmatrix} = \begin{bmatrix} 1 \cdot 1 \\ 1 \cdot 0 \\ 0 \cdot 1 \\ 0 \cdot 0 \end{bmatrix} = \begin{bmatrix} 1 \\ 0 \\ 0 \\ 0 \end{bmatrix}$$

2. $|01\rangle = |0\rangle \otimes |1\rangle$
   $$|01\rangle = \begin{bmatrix} 1 \\ 0 \end{bmatrix} \otimes \begin{bmatrix} 0 \\ 1 \end{bmatrix} = \begin{bmatrix} 1 \cdot 0 \\ 1 \cdot 1 \\ 0 \cdot 0 \\ 0 \cdot 1 \end{bmatrix} = \begin{bmatrix} 0 \\ 1 \\ 0 \\ 0 \end{bmatrix}$$

3. $|10\rangle = |1\rangle \otimes |0\rangle$
   $$|10\rangle = \begin{bmatrix} 0 \\ 1 \end{bmatrix} \otimes \begin{bmatrix} 1 \\ 0 \end{bmatrix} = \begin{bmatrix} 0 \cdot 1 \\ 0 \cdot 0 \\ 1 \cdot 1 \\ 1 \cdot 0 \end{bmatrix} = \begin{bmatrix} 0 \\ 0 \\ 1 \\ 0 \end{bmatrix}$$

4. $|11\rangle = |1\rangle \otimes |1\rangle$
   $$|11\rangle = \begin{bmatrix} 0 \\ 1 \end{bmatrix} \otimes \begin{bmatrix} 0 \\ 1 \end{bmatrix} = \begin{bmatrix} 0 \cdot 0 \\ 0 \cdot 1 \\ 1 \cdot 0 \\ 1 \cdot 1 \end{bmatrix} = \begin{bmatrix} 0 \\ 0 \\ 0 \\ 1 \end{bmatrix}$$

同样，我们也容易观察到其张量积结果与Diarc符号形式在二进制表示上的对应关系，这会方便我们使用和编码。


## 1.CNOT门
CNOT门是一个受控门，当控制位为$ |1\rangle $时，目标位进行X门操作，否则不操作。
其矩阵表示为
$$
CNOT = \begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 1 & 0
\end{bmatrix}
$$
对与任意量子态$|\psi\rangle = \begin{bmatrix} \alpha \\ \beta \\ \gamma \\ \delta \end{bmatrix}$，CNOT门的作用结果为：
$$|\psi'\rangle = CNOT|\psi\rangle = \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \end{bmatrix} \begin{bmatrix} \alpha \\ \beta \\ \gamma \\ \delta \end{bmatrix} = \begin{bmatrix} \alpha \\ \beta \\ \delta \\ \gamma \end{bmatrix}$$

将四个基态分别与CNOT矩阵的每一列按序对照，容易知道CNOT门的机制，并可以知道其并不创造新的相位和振幅

In [12]:
#实验1:基态测试
from pyqpanda3.core import *
circuit = QCircuit(1)
# 对量子位0应用 X 门，得到1态。并以其为控制比特，对量子位1应用 CNOT 门。
circuit << X(0) << CNOT(0, 1)

prog = QProg(circuit)
#注意，这里的控制比特是作为第0位的，也就是Dirac符号下最右边的低位。
prog << measure(0,0) << measure(1,1)

print(prog)

QVM = CPUQVM()
QVM.run(prog,4096)

print(QVM.result().get_counts())

circuit << CNOT(0, 1)
prog = QProg(circuit)
prog << measure(0,0) << measure(1,1)
print(prog)
QVM.run(prog,4096)
print(QVM.result().get_counts())




          ┌─┐         ┌─┐    
q_0:  |0>─┤X├ ───*── ─┤M├─── 
          └─┘ ┌──┴─┐  └╥┘┌─┐ 
q_1:  |0>──── ┤CNOT├ ──╫─┤M├ 
              └────┘   ║ └╥┘ 
 c :   / ══════════════╩══╩═
                        0  1


{'11': 4096}

          ┌─┐                ┌─┐    
q_0:  |0>─┤X├ ───*── ───*── ─┤M├─── 
          └─┘ ┌──┴─┐ ┌──┴─┐  └╥┘┌─┐ 
q_1:  |0>──── ┤CNOT├ ┤CNOT├ ──╫─┤M├ 
              └────┘ └────┘   ║ └╥┘ 
 c :   / ═════════════════════╩══╩═
                               0  1


{'01': 4096}


In [2]:
#实验2：控制位叠加态测试
#我们计划使用RX旋转门，如下
pi = 3.141592653589793
from pyqpanda3.core import *

circuit = QCircuit(0)
circuit << RX(0, 2*pi/6)
#这里使用2*theta是因为Bloch球和RX门的参数是theta/2。

prog = QProg(circuit)
QVM = CPUQVM()
QVM.run(prog,10240)

print(f"概率分布为{QVM.result().get_prob_dict()}")
prog << measure(0,0)
QVM.run(prog,10240)
print(f"测量结果为{QVM.result().get_prob_dict()}")




概率分布为{'0': 0.7500000000000001, '1': 0.24999999999999994}
测量结果为{'0': 0.75078125, '1': 0.24921875}


In [3]:
#下面我们运用CNOT门，将控制位0的叠加态测试结果作为控制位，对目标位1应用CNOT门。

circuit2 = QCircuit(1)
circuit2 << RX(0, 2*pi/6) << CNOT(0,1)

prog2 = QProg(circuit2)
prog2 << measure(0,0) << measure(1,1)
print(prog2)

QVM2 = CPUQVM()
QVM2.run(prog2,10240)

print(f"概率分布为{QVM2.result().get_prob_dict()}")



          ┌──────────────┐         ┌─┐    
q_0:  |0>─┤RX(1.04719755)├ ───*── ─┤M├─── 
          └──────────────┘ ┌──┴─┐  └╥┘┌─┐ 
q_1:  |0>───────────────── ┤CNOT├ ──╫─┤M├ 
                           └────┘   ║ └╥┘ 
 c :   / ═══════════════════════════╩══╩═
                                     0  1


概率分布为{'00': 0.74951171875, '11': 0.25048828125}


# 2.SWAP门
SWAP门的作用是交换两个量子态的状态。其矩阵表示如下
$$ SWAP = \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \end{bmatrix} $$

由二进制联系，我们可以很容易的看出基态的映射结果。
<div style="text-align: center;">
  <img src="Images/SWAP_Gate.png" alt="SWAP门示意图" style="width: 50%; max-width: 600px; height: auto;">
</div>



In [4]:
#SWAP测试，我们仍然使用特定的RX门生成叠加态来测试
pi = 3.141592653589793
from pyqpanda3.core import *

circuit = QCircuit(0)
circuit << RX(0, 2*pi/6)
#这里使用2*theta是因为Bloch球和RX门的参数是theta/2。

prog = QProg(circuit)
QVM = CPUQVM()
QVM.run(prog,10240)

print(f"概率分布为{QVM.result().get_prob_dict()}")
prog << measure(0,0)
QVM.run(prog,10240)
print(f"测量结果为{QVM.result().get_prob_dict()}")

概率分布为{'0': 0.7500000000000001, '1': 0.24999999999999994}
测量结果为{'0': 0.753515625, '1': 0.246484375}


In [9]:
#下面我们运用SWAP门，交换量子态的状态。

circuit2 = QCircuit(1)
circuit2 << RX(0, 2*pi/6) << SWAP(0,1)

prog2 = QProg(circuit2)
prog2 << measure(0,0) << measure(1,1)


QVM2 = CPUQVM()
QVM2.run(prog2,10240)

print(prog2)
print(f"运用SWAP门后的概率分布为{QVM2.result().get_prob_dict()}")

circuit = QCircuit(1)
circuit << RX(0, 2*pi/6) 

prog = QProg(circuit)
prog << measure(0,0) << measure(1,1)


QVM1 = CPUQVM()
QVM1.run(prog,10240)

print(prog)
print(f"不使用SWAP门的概率分布为{QVM1.result().get_prob_dict()}")


          ┌──────────────┐    ┌─┐    
q_0:  |0>─┤RX(1.04719755)├ X ─┤M├─── 
          └──────────────┘ │  └╥┘┌─┐ 
q_1:  |0>───────────────── X ──╫─┤M├ 
                               ║ └╥┘ 
 c :   / ══════════════════════╩══╩═
                                0  1


运用SWAP门后的概率分布为{'00': 0.7490234375, '10': 0.2509765625}

          ┌──────────────┐  ┌─┐ 
q_0:  |0>─┤RX(1.04719755)├ ─┤M├ 
          └┬─┬───────────┘  └╥┘ 
q_1:  |0>──┤M├──────────── ──╫─ 
           └╥┘               ║  
 c :   / ═══╩════════════════╩═
             1                0


不使用SWAP门的概率分布为{'00': 0.75068359375, '01': 0.24931640625}


# 3.CR门（受控旋转门）

CR门（Controlled Rotation gate）是一种受控门，当控制量子比特为 $|1\rangle$ 时，对目标量子比特进行旋转操作。其矩阵表示如下：

$$ CR(\theta) = \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & e^{i\theta} \end{bmatrix} $$

或者等价地表示为：

$$ CR(\theta) = \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & \cos\theta + i\sin\theta \end{bmatrix} $$

需要注意的是，CR门的作用是：当控制量子比特为 $|1\rangle$ 时，对目标量子比特应用相位旋转 $RZ(\theta)$，而不是RX门。

对于双量子比特态 $|\psi\rangle = \alpha|00\rangle + \beta|01\rangle + \gamma|10\rangle + \delta|11\rangle$，CR门的作用结果为：

$$|\psi'\rangle = CR(\theta)|\psi\rangle = \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & e^{i\theta} \end{bmatrix} \begin{bmatrix} \alpha \\ \beta \\ \gamma \\ \delta \end{bmatrix} = \begin{bmatrix} \alpha \\ \beta \\ \gamma \\ \delta e^{i\theta} \end{bmatrix}$$

其不影响观测结果，仅在控制比特为1时对某位进行相位调控


In [14]:
#CR门实验
from pyqpanda3.core import *
pi = 3.1415926

circuit1 = QCircuit()
circuit2 = QCircuit()

circuit1 << X(0) << CR(0,1,pi/6)
circuit2 << CR(0,1,pi/6)

prog1 = QProg(circuit1)
prog2 = QProg(circuit2)

prog1 << measure(0,0) << measure(1,1)
prog2 << measure(0,0) << measure(1,1)

QVM1 = CPUQVM()
QVM2 = CPUQVM()

QVM1.run(prog1,1024)
result1 = QVM1.result().get_prob_dict()
print("控制位为1时，目标位旋转结果为：",result1)

QVM2.run(prog2,1024)
result2 = QVM2.result().get_prob_dict()
print("控制位为0时，目标位旋转结果为：",result2)


控制位为1时，目标位旋转结果为： {'01': 1.0}
控制位为0时，目标位旋转结果为： {'11': 1.0}


In [3]:
4.

4.0